# HW8 Implementation

In [2]:
import numpy as np

In [3]:
P = np.array([
    [0, 3/4, 0, 1/4, 0],
    [3/4, 0, 0, 1/4, 0],
    [0, 1/2, 1/2, 0, 0],
    [0, 0, 0, 0, 1],
    [0, 1/2, 1/2, 0, 0]
])

# Calculating P^2 and P^3
P_squared = np.linalg.matrix_power(P, 2)
P_cubed = np.linalg.matrix_power(P, 3)

In [8]:
print(P_cubed)

[[0.       0.546875 0.125    0.140625 0.1875  ]
 [0.421875 0.125    0.125    0.140625 0.1875  ]
 [0.1875   0.40625  0.125    0.15625  0.125   ]
 [0.375    0.25     0.25     0.125    0.      ]
 [0.1875   0.40625  0.125    0.15625  0.125   ]]


In [12]:
3/16

0.1875

In [15]:
0.15625*64

10.0